In [ ]:
pip install transformers datasets accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
from google.colab import drive
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, AutoModelForSequenceClassification, AutoTokenizer,  EarlyStoppingCallback
from datasets import load_dataset, load_metric
import datasets
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
data = 'drive/MyDrive/Celtron/data/sentiment/balanced_dataframe.hf'

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

# Models

In [ ]:
model = BertForSequenceClassification.from_pretrained('csebuetnlp/banglabert', num_labels = 3)
model = model.to(device)
tokenizer = BertTokenizer.from_pretrained('csebuetnlp/banglabert')
# dataset = load_dataset(data)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/586 [00:00<?, ?B/s]

You are using a model of type electra to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at csebuetnlp/banglabert and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNor

tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'BertTokenizer'.


# Dataset

In [ ]:
# ds = dataset['train']

In [ ]:
# ds =ds.rename_column('Label','labels')

In [ ]:
# shuffled_data = ds.shuffle(seed = 42)

In [ ]:
# train_data = shuffled_data.select(range(15500))

In [ ]:
# test_data = ds[15500:]

# Prepare Data

In [ ]:
# ds = train_data.train_test_split(test_size = 0.2)

In [ ]:
# ds

In [ ]:
ds = datasets.load_from_disk(data)

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['data', 'labels'],
        num_rows: 15508
    })
    valid: Dataset({
        features: ['data', 'labels'],
        num_rows: 1939
    })
    test: Dataset({
        features: ['data', 'labels'],
        num_rows: 1939
    })
})

In [ ]:
def preprocess_function(examples):
   return tokenizer(examples["data"], max_length=512, padding = 'max_length', truncation=True)

In [ ]:
tokenized_train = ds['train'].map(preprocess_function, batched=True)
tokenized_test = ds['valid'].map(preprocess_function, batched=True)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
import os
dir = 'output'
if not os.path.exists(dir):
  os.mkdir('output')

In [ ]:
os.listdir('./output/')

[]

# Load pretrained model

In [ ]:
model = torch.load('drive/MyDrive/Celtron/data/sentiment/models/bert_buet_v2.pth', map_location=device)

In [ ]:
# for name, param in model.named_parameters():
#   if param.requires_grad:
#       print(name, param.data)

# Train

In [ ]:
training_args = TrainingArguments(
   output_dir='./output/',
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   do_train = True,
   num_train_epochs=5,
   load_best_model_at_end = True,
   evaluation_strategy = 'epoch',
   save_strategy="epoch"
)

In [ ]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   callbacks = [EarlyStoppingCallback(1)]
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.109200,1.108931
2,1.101100,1.107363


Epoch,Training Loss,Validation Loss
1,1.109200,1.108931
2,1.101100,1.107363
3,1.106000,1.099835


In [ ]:
torch.save(model, 'drive/MyDrive/Celtron/data/sentiment/models/bert_buet_v3.pth')

In [ ]:
idx = 119
text = test_data['Data'][idx]
label = test_data['labels'][idx]
print(f"Text: {text}")

Text: গুটি করতে অসছে । ইতিহাস সাক্ষী এ জাতীয়তাবাদ অথবা ধর্ম নিরপেক্ষতা থেকেই বাংলাদেশের জন্ম । যেখানে একই ধর্মের পূর্ব পাকিস্তান ও পশ্চিম পাকিস্তান ছিল


In [ ]:
token = tokenizer(text, return_tensors = 'pt')
token = token.to(device)
output = model(**token)
print(f"Output: {output.logits}")
pred = torch.argmax(output.logits)
print(f"Prediction: {pred}\nActual: {label}")
if pred == 0:
  print('Neutral')
elif pred == 1:
  print('Positive')
else:
  print('Negative')

Output: tensor([[0.0633, 0.8162, 0.6794]], device='cuda:0', grad_fn=<AddmmBackward0>)
Prediction: 1
Actual: 1
Positive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Celtron/ai_models/

In [ ]:
torch.save(model, 'banglabert_sentiment_new.pth')

# Inference

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

In [ ]:
os.listdir('/content/drive/MyDrive/Celtron/ai_models/sentiment_analysis/')

['banglabert_sentiment.pth',
 'en_model_sentiment.pth',
 'bn_bert_sagor.pkl',
 'en_distil_bert.pkl']

In [ ]:
%cd /content/drive/MyDrive/Celtron/ai_models/sentiment_analysis/

/content/drive/MyDrive/Celtron/ai_models/sentiment_analysis


In [ ]:
bn_model = torch.load('banglabert_sentiment.pth')
bn_tokenizer = BertTokenizer.from_pretrained('sagorsarker/bangla-bert-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

In [ ]:
text = '''
সফল হোক
'''

In [ ]:
token = bn_tokenizer(text, return_tensors = 'pt')
token = token.to(device)

In [ ]:
out = bn_model(**token)
pred = torch.argmax(out.logits)
print(out.logits)
print(pred)

if pred == 0:
  print('Neutral')
elif pred == 1:
  print('Positive')
else:
  print('Negative')

tensor([[ 1.9216,  1.2515, -3.5686]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
tensor(0, device='cuda:0')
Neutral


In [ ]:
text = test_data['Data'][1:]
label = test_data['labels'][1:]

acc = 0
total = len(text)

for idx in range(len(text)):
  text_input = text[idx]
  actual_label = label[idx]
  token = bn_tokenizer(text_input, return_tensors = 'pt')
  token = token.to(device)
  output = bn_model(**token)
  pred_label = torch.argmax(output.logits)
  if pred_label == actual_label:
    acc += 1
print(f"Accuracy: {acc/total}")

Accuracy: 0.9427312775330396


In [ ]:
print(label)

[1, 0, 2, 1, 1, 0, 0, 2, 0, 2, 0, 1, 0, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 0, 2, 1, 1, 1, 0, 2, 2, 1, 2, 0, 2, 2, 1, 0, 2, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 2, 2, 2, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 1, 2, 1, 0, 2, 1, 0, 1, 0, 2, 2, 2, 1, 2, 1, 2, 1, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 1, 0, 1, 0, 0, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 0, 2, 2, 1, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2, 1, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 1, 1, 1, 1, 0, 2, 0, 1, 2, 0, 2, 0, 0, 0, 2, 0, 1, 1, 0, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 0, 1, 2, 1, 0, 1, 2, 2, 1, 1, 1, 2, 1, 0, 2, 1, 1, 2, 0, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 0]


In [ ]:
from collections import Counter
element_counts = Counter(label)
print(element_counts)

Counter({2: 93, 1: 82, 0: 52})


In [ ]:
print(test_data['labels'])

[1, 1, 0, 2, 1, 1, 0, 0, 2, 0, 2, 0, 1, 0, 2, 1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 1, 0, 2, 1, 1, 1, 0, 2, 2, 1, 2, 0, 2, 2, 1, 0, 2, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 2, 2, 2, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 1, 2, 1, 0, 2, 1, 0, 1, 0, 2, 2, 2, 1, 2, 1, 2, 1, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2, 1, 0, 1, 0, 0, 2, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 0, 2, 2, 1, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2, 1, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 2, 0, 0, 0, 1, 1, 1, 1, 0, 2, 0, 1, 2, 0, 2, 0, 0, 0, 2, 0, 1, 1, 0, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 0, 1, 2, 1, 0, 1, 2, 2, 1, 1, 1, 2, 1, 0, 2, 1, 1, 2, 0, 2, 1, 2, 2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 0, 1, 0, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 0]


In [ ]:
example_text = 'এরা পারেও বটে ☺ আরে ভাই আমি তো ক্যাপশন দেখে ভয়ই পাইছিলাম , আসল বিষয় বুঝলাম ভিডিওটা দেখে'
token = bn_tokenizer(example_text, return_tensors = 'pt')
print(token)

{'input_ids': tensor([[  101,  3609,  2083,  2058,  5552,   100,  2070,  2094,  2570,  2169,
          4058,  2395,  4134,  7724,  5829,  5212,  2465, 14475,  9294,  2062,
         87414,  9304,  1011,  4837,  8136,  9294,  5740, 18679,  9304, 67355,
          2465,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
example = token.input_ids
example = example.to(device)
with torch.no_grad():
  output = model(example)
  print(output.logits)

tensor([[-0.1077,  0.0422, -0.2491]], device='cuda:0')


In [ ]:
pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 51.2 MB/s eta 0:00:00


In [ ]:
torch.onnx.export(bn_model, example, 'bn_model_onnx.onnx' ,verbose = True)

In [ ]:
quantized_model = torch.quantization.quantize_dynamic(model = bn_model)

In [ ]:
text = test_data['Data'][1:]
label = test_data['labels'][1:]

acc = 0
total = len(text)

for idx in range(len(text)):
  text_input = text[idx]
  actual_label = label[idx]
  token = bn_tokenizer(text_input, return_tensors = 'pt').input_ids
  token = token.to(device)
  output = bn_model(token)
  pred_label = torch.argmax(output.logits)
  if pred_label == actual_label:
    acc += 1
print(f"Accuracy: {acc/total}")

Accuracy: 0.9383259911894273


In [ ]:
torch.save(quantized_model, "quantized_bn_model.pth")

# English

In [ ]:
en_model = AutoModelForSequenceClassification.from_pretrained('lxyuan/distilbert-base-multilingual-cased-sentiments-student', num_labels = 3)
en_model = en_model.to(device)
en_tokenizer = AutoTokenizer.from_pretrained('lxyuan/distilbert-base-multilingual-cased-sentiments-student')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/541M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
text = '''
We’ve been dealing with invisibility. We started realizing we’re kind of fading. So many of our friends say that: that they’re becoming invisible. Everybody needs a welcome from somebody else so that they can feel useful. It’s a real source of energy. And when you realize you’re not getting that as much—what happens is you get scared. And you also say: maybe we could do something a little different. So at some point we came up with the idea of the cute older couple. We were hoping to find some younger friends. We’ve always been attracted to younger people. You know, young people struggle. So we like to support them and wish them well and give them a lot of approval. And young people need cute old couples. They love cute old couples. So we decided to play it up a little bit. That’s what it is: ‘play.’ It’s really play. Have you ever seen two dogs greet each other? One dog will drop down, and bam, suddenly they’re playing. I think that’s what we’re doing. We’re inviting a play response.
'''
token = en_tokenizer(text, return_tensors = 'pt')
token = token.to(device)
output = en_model(**token)
pred = torch.argmax(output.logits)
if pred == 0:
  print('Positive')
elif pred == 1:
  print('Neutral')
else:
  print('Negative')

Positive


In [ ]:
torch.save(en_model, 'en_model_sentiment.pth')

In [ ]:
quantized_model = torch.quantization.quantize_dynamic(model = en_model)
quantized_model = quantized_model.to(device)

In [ ]:
text = '''
We’ve been dealing with invisibility. We started realizing we’re kind of fading. So many of our friends say that: that they’re becoming invisible. Everybody needs a welcome from somebody else so that they can feel useful. It’s a real source of energy. And when you realize you’re not getting that as much—what happens is you get scared. And you also say: maybe we could do something a little different. So at some point we came up with the idea of the cute older couple. We were hoping to find some younger friends. We’ve always been attracted to younger people. You know, young people struggle. So we like to support them and wish them well and give them a lot of approval. And young people need cute old couples. They love cute old couples. So we decided to play it up a little bit. That’s what it is: ‘play.’ It’s really play. Have you ever seen two dogs greet each other? One dog will drop down, and bam, suddenly they’re playing. I think that’s what we’re doing. We’re inviting a play response.
'''
token = en_tokenizer(text, return_tensors = 'pt')
token = token.to(device)
output = quantized_model(**token)
pred = torch.argmax(output.logits)
if pred == 0:
  print('Positive')
elif pred == 1:
  print('Neutral')
else:
  print('Negative')

NotImplementedError: Could not run 'quantized::linear_dynamic' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::linear_dynamic' is only available for these backends: [CPU, Meta, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

CPU: registered at ../aten/src/ATen/native/quantized/cpu/qlinear_dynamic.cpp:662 [

In [ ]:
torch.save(quantized_model, "quantized_en_model.pth")